In [1]:
import pandas as pd

In [2]:
lang = 'heb'

In [3]:
path = "/pscratch/sd/c/chubakov/code/morphology/gpu/neural-transducer/model/sigmorphon17-task1/large/monotag-hmmfull/{lang}-high.decode.dev.tsv".format(lang=lang)
path

'/pscratch/sd/c/chubakov/code/morphology/gpu/neural-transducer/model/sigmorphon17-task1/large/monotag-hmmfull/heb-high.decode.dev.tsv'

In [4]:
df = pd.read_csv(path, sep='\t')

In [5]:
df

,prediction,target,loss,dist
0,א ַ ל ת ְ ּ א ַ ח ֵ ר !,א ַ ל ת ְ ּ א ַ ח ֵ ר !,8.273608e-05,0
1,ל ֹ א א ִ ח ַ ר ְ ת ִ ּ י ?,ל ֹ א א ִ ח ַ ר ְ ת ִ ּ י ?,1.100270e-06,0
2,א ִ ח ַ ר ְ ת ֶ ּ ן .,א ִ ח ַ ר ְ ת ֶ ּ ן .,6.750402e-06,0
3,א ַ ל ת ְ ּ א ַ ח ֲ ר ִ י !,א ַ ל ת ְ ּ א ַ ח ֲ ר ִ י !,4.012843e-06,0
4,ל ֹ א ת ְ ּ א ַ ח ֵ ר .,ל ֹ א ת ְ ּ א ַ ח ֵ ר .,3.654822e-05,0
...,...,...,...,...
995,א ַ ל ת ְ ּ ר ֻ פ ְ ּ ד ו ּ !,א ַ ל ת ְ ּ ר ֻ פ ְ ּ ד ו ּ !,5.591801e-06,0
996,א ַ ל ת ְ ּ ר ֻ פ ַ ּ ד ְ נ ָ ה !,א ַ ל ת ְ ּ ר ֻ פ ַ ּ ד ְ נ ָ ה !,7.036624e-06,0
997,ל ֹ א ר ֻ פ ַ ּ ד ְ ת ָ ּ .,ל ֹ א ר ֻ פ ַ ּ ד ְ ת ָ ּ .,7.694394e-07,0
998,ר ֻ פ ַ ּ ד ְ ת ֶ ּ ם .,ר ֻ פ ַ ּ ד ְ ת ֶ ּ ם .,4.273263e-06,0


In [6]:
def fix_spacing(sentence):
    
    sentence = sentence.replace("   ", "_")
    sentence = sentence.replace(" ", "")
    sentence = sentence.replace("_", " ")
    
    return sentence
    
df['prediction'] = df.apply(lambda x: fix_spacing(x.prediction), axis=1)
df['target'] = df.apply(lambda x: fix_spacing(x.target), axis=1)

In [7]:
df

,prediction,target,loss,dist
0,אַל תְּאַחֵר!,אַל תְּאַחֵר!,8.273608e-05,0
1,לֹא אִחַרְתִּי?,לֹא אִחַרְתִּי?,1.100270e-06,0
2,אִחַרְתֶּן.,אִחַרְתֶּן.,6.750402e-06,0
3,אַל תְּאַחֲרִי!,אַל תְּאַחֲרִי!,4.012843e-06,0
4,לֹא תְּאַחֵר.,לֹא תְּאַחֵר.,3.654822e-05,0
...,...,...,...,...
995,אַל תְּרֻפְּדוּ!,אַל תְּרֻפְּדוּ!,5.591801e-06,0
996,אַל תְּרֻפַּדְנָה!,אַל תְּרֻפַּדְנָה!,7.036624e-06,0
997,לֹא רֻפַּדְתָּ.,לֹא רֻפַּדְתָּ.,7.694394e-07,0
998,רֻפַּדְתֶּם.,רֻפַּדְתֶּם.,4.273263e-06,0


In [8]:
df['clause_match'] = df['prediction'] == df['target']
df["clause_match"] = df["clause_match"].astype(int)

In [9]:
def word_level_cmp(prediction, target):    
    
    words1 = prediction.split(' ')
    words2 = target.split(' ')
    
    # print(len(words1), len(words2))
    
    num_correct = 0
    
    min_l = min(len(words1), len(words2))
    
    for i in range(min_l):
        if words1[i] == words2[i]:
            num_correct += 2
            
    return num_correct / (len(words1) + len(words2))    

In [10]:
df['word_match'] = df.apply(lambda x: word_level_cmp(x.prediction, x.target), axis=1)

In [11]:
df

,prediction,target,loss,dist,clause_match,word_match
0,אַל תְּאַחֵר!,אַל תְּאַחֵר!,8.273608e-05,0,1,1.0
1,לֹא אִחַרְתִּי?,לֹא אִחַרְתִּי?,1.100270e-06,0,1,1.0
2,אִחַרְתֶּן.,אִחַרְתֶּן.,6.750402e-06,0,1,1.0
3,אַל תְּאַחֲרִי!,אַל תְּאַחֲרִי!,4.012843e-06,0,1,1.0
4,לֹא תְּאַחֵר.,לֹא תְּאַחֵר.,3.654822e-05,0,1,1.0
...,...,...,...,...,...,...
995,אַל תְּרֻפְּדוּ!,אַל תְּרֻפְּדוּ!,5.591801e-06,0,1,1.0
996,אַל תְּרֻפַּדְנָה!,אַל תְּרֻפַּדְנָה!,7.036624e-06,0,1,1.0
997,לֹא רֻפַּדְתָּ.,לֹא רֻפַּדְתָּ.,7.694394e-07,0,1,1.0
998,רֻפַּדְתֶּם.,רֻפַּדְתֶּם.,4.273263e-06,0,1,1.0


In [12]:
print('current language: ', lang)

current language:  heb


In [13]:
df.describe()

,loss,dist,clause_match,word_match
count,1.000000e+03,1000.000000,1000.000000,1000.000000
mean,3.118177e-02,1.251000,0.804000,0.918194
std,7.913212e-02,3.463812,0.397167,0.184557
min,5.431521e-07,0.000000,0.000000,0.000000
25%,2.198787e-05,0.000000,1.000000,1.000000
50%,1.391014e-04,0.000000,1.000000,1.000000
75%,4.224008e-03,0.000000,1.000000,1.000000
max,6.592913e-01,25.000000,1.000000,1.000000


In [14]:
out_path = "/pscratch/sd/c/chubakov/code/morphology/gpu/neural-transducer/output-lstm/{lang}-output.dev.tsv".format(lang=lang)
out_path

df.to_csv(out_path, sep='\t')

In [15]:
eval_out_path = "/pscratch/sd/c/chubakov/code/morphology/evaluation/lstm/res/{lang}.dev".format(lang=lang)
eval_out_path

eval_df = df[['prediction']]

eval_df.to_csv(eval_out_path, index=False, header=False, sep='\t')